In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../zkstats/core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.div(1.0,torch.mean(torch.div(1.0,dummy_data_tensor)))

In [5]:
# Verifier/ data consumer side:
# since have division, scale must be very LARGE, cant use just 0
# So, we allow error rate to be 10% instead of 1%
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs((self.w*torch.sum(torch.div(1.0,X)))-X.size()[1])<0.1*X.size()[1], self.w)
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [9]:
# prover calculates settings, send to verifier
theory_output = torch.div(1.0,torch.mean(torch.div(1.0,data_tensor)))
print("Theory_output: ",theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs((self.w*torch.sum(torch.div(1.0,X)))-X.size()[1])<0.1*X.size()[1], self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [6], "resources", settings_path)

Theory_output:  tensor(45.2144)
==== Generate & Calibrate Setting ====
scale:  [6]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":6,"param_scale":6,"scale_rebase_multiplier":10,"lookup_range":[-61368,184392],"logrows":18,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":907,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,6],"model_input_scales":[6],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Recip":{"scale":4096.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [10]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 20.96218991279602 seconds
Theory output:  tensor(45.2144)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 45.21875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[11712583131456230033, 7998639690917952521, 17628064685104548320, 3364894255067083796], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10409296726488761395, 15557203226960814343, 16692342436085548322, 2420882828764455196]]], 'proof': '03cea941df4d7c87c5e6c7f498d12d876b2699bf318b4618c9dbef4573dba4cc0d8cdb3e5e1186fdc6226f09984c65f37e8e54aa81ef4a846071bfb735dc58ae075e30a46baa9a933b90e13b5a94fc368f0255bd179b4f593e46871dc847497a0f05ba0945f380ab52aac4ce20fe20e32cd9af9e452cd3c899d16b3fe6aa0c7814eeecb23bcf005558ca151a6992b74cc75ae58b3f961eb037a1314cb3d8b4140750167418e16c28101d774ce1a63dcc870bfe784b2c3d4501274601f650d2b316dc2a7bfebe5335453ccb510c579cc14958f58f20db29723ab7ad2dd4b42a7f2e681e83a07d537951d4a1b54347a103a15406bf9ce9ea200dd74dded56c0b9e0d24bacf085a8db98ad5135dc329e939205c1b0db8a4a8a5d9e8327ad55a986b14e04c01d945244acc257514633b347c294548717672730c885122885916c6940ba3ffdf621886c767a66b0b56038d55c7f1a0f0d98ed2854e34c8f0e3a2

In [11]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[11712583131456230033, 7998639690917952521, 17628064685104548320, 3364894255067083796], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10409296726488761395, 15557203226960814343, 16692342436085548322, 2420882828764455196]]]
proof boolean:  1.0
proof result 1 : 45.21875
verified
